In [2]:
import pandas as pd
#df=pd.read_csv('https://cloud.minsa.gob.pe/s/Y8w3wHsEdYQSZRp/download', sep=';', usecols=['FECHA_CORTE', 'EDAD', 'SEXO', 'FECHA_RESULTADO', 'METODODX', 'DEPARTAMENTO'], parse_dates=['FECHA_RESULTADO'])
df=pd.read_csv('positivos_covid.csv', sep=';', usecols=['FECHA_CORTE', 'EDAD', 'SEXO', 'FECHA_RESULTADO', 'METODODX', 'DEPARTAMENTO'], parse_dates=['FECHA_RESULTADO'])
fecha_corte=df['FECHA_CORTE'].drop_duplicates().set_axis(['fecha_corte'])
fecha_corte.to_json("../resultados/fecha_corte_positivos.json")

In [3]:
# DIARIO AG, PCR, PR
df_positivos=df[['FECHA_RESULTADO','METODODX','SEXO']].groupby(['FECHA_RESULTADO','METODODX']).count()
df_positivos=df_positivos.reset_index()
df_positivos=df_positivos.pivot(index='FECHA_RESULTADO', columns='METODODX', values='SEXO')
df_positivos=df_positivos.rename_axis(None, axis=1)
df_positivos=df_positivos.fillna(0).astype('int')
df_positivos

,AG,PCR,PR
FECHA_RESULTADO,,,
2020-03-06,0,1,0
2020-03-07,0,5,0
2020-03-08,0,2,0
2020-03-09,0,3,0
2020-03-10,0,1,0
...,...,...,...
2021-05-18,2887,1702,661
2021-05-19,2738,1413,545
2021-05-20,2463,1349,474


In [4]:
# ACUMULADO AG, PCR, PR
df_positivos_cum=df_positivos.cumsum()
df_positivos_cum

,AG,PCR,PR
FECHA_RESULTADO,,,
2020-03-06,0,1,0
2020-03-07,0,6,0
2020-03-08,0,8,0
2020-03-09,0,11,0
2020-03-10,0,12,0
...,...,...,...
2021-05-18,423765,570708,913930
2021-05-19,426503,572121,914475
2021-05-20,428966,573470,914949


In [5]:
col_poblacion=[426806,
1180638,
430736,
1497438,
668213,
1453711,
1129854,
1357075,
365317,
760267,
975182,
1361467,
2016771,
1310785,
10628470,
1027559,
173811,
192740,
271904,
2047954,
1237997,
899648,
370974,
251521,
589110]

In [16]:
# ACUMULADO POR DEPARTAMENTO
df_positivos_departamento=df[['DEPARTAMENTO','METODODX']]
def function(valueX):
  if 'LIMA REGION' in valueX:
    return 'LIMA'
  else:
    return valueX
df_positivos_departamento['DEPARTAMENTO']=df_positivos_departamento['DEPARTAMENTO'].map(function)
df_positivos_departamento=df_positivos_departamento.groupby(['DEPARTAMENTO']).count()
df_positivos_departamento['POBLACION']=col_poblacion
df_positivos_departamento['INDICE']=round(df_positivos_departamento['METODODX']/(df_positivos_departamento['POBLACION']/100000)).astype('int')
df_positivos_departamento

<ipython-input-16-0d0b6f5d0e35>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_positivos_departamento['DEPARTAMENTO']=df_positivos_departamento['DEPARTAMENTO'].map(function)


,METODODX,POBLACION,INDICE
DEPARTAMENTO,,,
AMAZONAS,28174,426806,6601
ANCASH,68378,1180638,5792
APURIMAC,22469,430736,5216
AREQUIPA,80787,1497438,5395
AYACUCHO,28392,668213,4249
CAJAMARCA,57238,1453711,3937
CALLAO,94469,1129854,8361
CUSCO,60988,1357075,4494
HUANCAVELICA,14407,365317,3944


In [1]:
# ACUMULADO POR GRUPO ETARIO
bins = [18,20,30,40,50,60,70,80,df['EDAD'].max()+1]
labels = ['18 a 19 años','20 a 29 años','30 a 39 años','40 a 49 años','50 a 59 años','60 a 69 años','70 a 79 años','80 años a más']
poblacion_por_grupo_etario = [900000,5700000,5300000,4400000,3300000,2300000,1300000,700000] 
df_positivos_edades = df
df_positivos_edades['GRUPO_ETARIO'] = pd.cut(df['EDAD'], bins=bins, labels=labels, right=False)
df_positivos_edades = df_positivos_edades.groupby(['GRUPO_ETARIO'])["METODODX"].count().reset_index()
df_positivos_edades.rename(columns = {'METODODX':'POSITIVOS'}, inplace = True)
df_positivos_edades['POBLACION']=poblacion_por_grupo_etario
df_positivos_edades['PORCENTAJE']=round(df_positivos_edades['POSITIVOS']/df_positivos_edades['POBLACION']*100,2)
df_positivos_edades=df_positivos_edades.set_index('GRUPO_ETARIO')
df_positivos_edades

NameError: name 'df' is not defined

In [6]:
df_positivos.to_csv('../resultados/positivos_diarios.csv')
df_positivos_cum.to_csv('../resultados/positivos_acumulados.csv')
df_positivos_departamento.to_csv('../resultados/positivos_por_departamentos.csv')
df_positivos_edades.to_csv('../resultados/positivos_por_edades.csv')